### In this notebook, we generate the results of the reddit-5K experiment of the article with precomputed persistence diagrams obtained from the authors code.

### Do not forget to move the .so files to the dist-packages repo of your python version! 

In [2]:
import numpy              as np
np.set_printoptions(threshold='nan')
import math       
import sys
from   random             import shuffle   
import matplotlib.pyplot  as plt
import matplotlib.image   as mpimg
import h5py

import tensorflow as tf
import _persistence_vector_grad
import _persistence_vector_essential_grad
persistence_vector_module = tf.load_op_library('persistence_vector.so')
persistence_vector_essential_module = tf.load_op_library('persistence_vector_essential.so')

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Data reading
This code assumes that precomputed persistence diagrams are available in a h5 file called reddit_5K.h5 in a repo called reddit_5K. This file can be obtained by running code provided by the authors, or by computing the diagrams manually with Gudhi using the notebook "Persistence Diagram Computations for Graphs". 

In [3]:
f = h5py.File("reddit_5K/reddit_5K.h5")
num_labels = 5

D0     = []
C0     = []
D0i    = []
C0i    = []
D1i    = []
C1i    = []
labels = []

for j in range(1,6):
    
    for index in f['data_views']['DegreeVertexFiltration_dim_0'][str(j)].keys():

        diag0  = f['data_views']['DegreeVertexFiltration_dim_0'][str(j)][index]
        diag0i = f['data_views']['DegreeVertexFiltration_dim_0_essential'][str(j)][index]
        diag1i = f['data_views']['DegreeVertexFiltration_dim_1_essential'][str(j)][index]
        num0  = diag0.shape[0]
        num0i = diag0i.shape[0]
        num1i = diag1i.shape[0]
        
        if(num0 > 0 and num0i > 0 and num1i > 0):
            D0.append(diag0)
            C0.append(num0)
            D0i.append(diag0i[:,0])
            C0i.append(num0i)
            D1i.append(diag1i[:,0])
            C1i.append(num1i)

            vector_lab = np.zeros(num_labels)
            vector_lab[j-1] = 1
            labels.append(vector_lab)
                
num_pts = len(labels)

## Definition of the network as described in the article

In [13]:
# Network parameters as decribed in the article
num_gaussians0  = 150
num_gaussians0i = 50
num_gaussians1i = 50
keep_prob       = 0.9
nu_tensor0      = tf.Variable([[0.01]], trainable=False)

# Random initialization of Gaussians
mu0     = tf.Variable(tf.random_uniform([num_gaussians0,2]))
sigma0  = tf.Variable(np.float32(3*np.ones([num_gaussians0,2])))
Gauss0  = tf.concat([mu0,sigma0],1)

mu0i     = tf.Variable(tf.random_uniform([num_gaussians0i,1]))
sigma0i  = tf.Variable(np.float32(3*np.ones([num_gaussians0i,1])))
Gauss0i  = tf.concat([mu0i,sigma0i],1)

mu1i     = tf.Variable(tf.random_uniform([num_gaussians1i,1]))
sigma1i  = tf.Variable(np.float32(3*np.ones([num_gaussians1i,1])))
Gauss1i  = tf.concat([mu1i,sigma1i],1)

# Random initialization for layer weights
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=1.0)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(1.0, shape=shape)
    return tf.Variable(initial)

# Placeholders
data0       = tf.placeholder(tf.float32, shape=[None, 2])
data0i      = tf.placeholder(tf.float32, shape=[None, 1])
data1i      = tf.placeholder(tf.float32, shape=[None, 1])
card0       = tf.placeholder(tf.float32, shape=[None, 1])
card0i      = tf.placeholder(tf.float32, shape=[None, 1])
card1i      = tf.placeholder(tf.float32, shape=[None, 1])
label       = tf.placeholder(tf.float32, shape=[None, num_labels])

# Treatment of the ordinary persistence diagram in dim 0
# Linear -> Batch Norm -> Dropout -> Linear -> ReLu -> Dropout 
v0  = persistence_vector_module.persistence_vector(data0, card0, Gauss0, nu_tensor0)
W01 = weight_variable([150,75])  
b01 = bias_variable([75])
layer01   = tf.matmul(v0,W01) + b01
mean01, variance01 = tf.nn.moments(layer01, axes=[0], keep_dims=True)
layer02   = tf.nn.batch_normalization(layer01, mean01, variance01, None, None, 1e-10)
layer03   = tf.nn.dropout(layer02, keep_prob)
W04 = weight_variable([75,75])  
b04 = bias_variable([75])
layer04   = tf.matmul(layer03,W04) + b04
layer05   = tf.nn.relu(layer04)
layer06   = tf.nn.dropout(layer05, keep_prob)

# Treatment of the essential persistence diagram in dim 0
# Linear -> Batch Norm -> Dropout -> Linear -> ReLu -> Dropout 
v0i = persistence_vector_essential_module.persistence_vector_essential(data0i, card0i, Gauss0i)
W0i1 = weight_variable([50,25])  
b0i1 = bias_variable([25])
layer0i1   = tf.matmul(v0i,W0i1) + b0i1
mean0i1, variance0i1 = tf.nn.moments(layer0i1, axes=[0], keep_dims=True)
layer0i2   = tf.nn.batch_normalization(layer0i1, mean0i1, variance0i1, None, None, 1e-10)
layer0i3   = tf.nn.dropout(layer0i2, keep_prob)
W0i4 = weight_variable([25,25])  
b0i4 = bias_variable([25])
layer0i4   = tf.matmul(layer0i3,W0i4) + b0i4
layer0i5   = tf.nn.relu(layer0i4)
layer0i6   = tf.nn.dropout(layer0i5, keep_prob)

# Treatment of the essential persistence diagram in dim 1
# Linear -> Batch Norm -> Dropout -> Linear -> ReLu -> Dropout 
v1i = persistence_vector_essential_module.persistence_vector_essential(data1i, card1i, Gauss1i)
W1i1 = weight_variable([50,25])  
b1i1 = bias_variable([25])
layer1i1   = tf.matmul(v1i,W1i1) + b1i1
mean1i1, variance1i1 = tf.nn.moments(layer1i1, axes=[0], keep_dims=True)
layer1i2   = tf.nn.batch_normalization(layer1i1, mean1i1, variance1i1, None, None, 1e-10)
layer1i3   = tf.nn.dropout(layer1i2, keep_prob)
W1i4 = weight_variable([25,25])  
b1i4 = bias_variable([25])
layer1i4   = tf.matmul(layer1i3,W1i4) + b1i4
layer1i5   = tf.nn.relu(layer1i4)
layer1i6   = tf.nn.dropout(layer1i5, keep_prob)

# Merge and treat the three previous branches
# Linear -> Batch Norm -> ReLu -> Linear -> Batch Norm -> Dropout -> ReLu -> Linear -> Batch Norm -> ReLu -> Linear -> Batch Norm
layer7 = tf.concat([layer06,layer0i6,layer1i6],1)
W8 = weight_variable([125,200])  
b8 = bias_variable([200])
layer8   = tf.matmul(layer7,W8) + b8
mean8, variance8 = tf.nn.moments(layer8, axes=[0], keep_dims=True)
layer9   = tf.nn.batch_normalization(layer8, mean8, variance8, None, None, 1e-10)
layer10  = tf.nn.relu(layer9)
W11 = weight_variable([200,100])  
b11 = bias_variable([100])
layer11   = tf.matmul(layer10,W11) + b11
mean11 ,variance11 = tf.nn.moments(layer11, axes=[0], keep_dims=True)
layer12   = tf.nn.batch_normalization(layer11, mean11, variance11, None, None, 1e-10)
layer13   = tf.nn.dropout(layer12, keep_prob)
layer14   = tf.nn.relu(layer13)
W15 = weight_variable([100,50])  
b15 = bias_variable([50])
layer15   = tf.matmul(layer14,W15) + b15
mean15, variance15 = tf.nn.moments(layer15, axes=[0], keep_dims=True)
layer16   = tf.nn.batch_normalization(layer15, mean15, variance15, None, None, 1e-10)
layer17   = tf.nn.relu(layer16)
W18 = weight_variable([50,5])  
b18 = bias_variable([5])
layer18   = tf.matmul(layer17,W18) + b18
mean18, variance18 = tf.nn.moments(layer18, axes=[0], keep_dims=True)
layer19   = tf.nn.batch_normalization(layer18, mean18, variance18, None, None, 1e-10)

## Shuffle Data

In [14]:
# Shuffle data so as to be homogeneous
C = list(zip(D0,C0,D0i,C0i,D1i,C1i,labels))
shuffle(C)
D0,C0,D0i,C0i,D1i,C1i,labels = zip(*C)

## Do the optimization and evaluate on dataset

In [17]:
# Set sub_ratio to x > 1 if you want to train and test on a sub dataset
sub_ratio      = 1

# Use 90% of data as training
training_ratio = 0.9
training_size  = np.round(training_ratio*num_pts/sub_ratio).astype(np.int)

# Split data into batches of size 128 and optimize over 30 epochs 
batch_size     = 128
nb_epoch       = 30

# Compute number of batches
if training_size % batch_size == 0:
    num_batches = training_size/batch_size
else:
    num_batches = training_size/batch_size + 1 
    
# Define accuracy
correct_prediction    = tf.equal(tf.argmax(layer19, 1), tf.argmax(label, 1))
accuracy              = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Optimize cross entropy loss with Gradient Descent and decaying learning rate
cross_entropy         = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=label, logits=layer19))
global_step           = tf.Variable(0, trainable=False)
learning_rate         = tf.train.exponential_decay(0.1, global_step, 25*num_batches, 0.5, staircase=True)
learning              = tf.train.GradientDescentOptimizer(learning_rate)
learning_step         = learning.minimize(cross_entropy, global_step=global_step)
gradmu0               = learning.compute_gradients(cross_entropy, mu0)

# Define training batches
chunked_data = []

for i in range(0,training_size,batch_size):
    
    if (training_size < i + batch_size):
        num_pts_in_batch = training_size - i
    else:
        num_pts_in_batch = batch_size
    
    batch_labels      = []        
    batch_data0       = []
    batch_card0       = []
    batch_data0i      = []
    batch_card0i      = []
    batch_data1i      = []
    batch_card1i      = []
    for j in range(num_pts_in_batch):
        batch_labels.append(labels[i+j])
        batch_data0.append(D0[i+j])
        batch_card0.append(C0[i+j])
        batch_data0i.append(D0i[i+j])
        batch_card0i.append(C0i[i+j])
        batch_data1i.append(D1i[i+j])
        batch_card1i.append(C1i[i+j])
        
    # Reshape data for dimensionality agreement with network 
    chunked_data.append((batch_labels, np.concatenate(batch_data0, 0), 
                                       np.reshape(batch_card0, (-1,1)), 
                                       np.reshape(np.concatenate(batch_data0i,0),(-1,1)), 
                                       np.reshape(batch_card0i,(-1,1)),
                                       np.reshape(np.concatenate(batch_data1i,0),(-1,1)),
                                       np.reshape(batch_card1i,(-1,1))))

# Define test set
test_labels      = []        
test_data0       = []
test_card0       = []
test_data0i      = []
test_card0i      = []
test_data1i      = []
test_card1i      = []
for i in range(training_size,num_pts/sub_ratio):      
    test_labels.append(labels[i])
    test_data0.append(D0[i])
    test_card0.append(C0[i])
    test_data0i.append(D0i[i])
    test_card0i.append(C0i[i])
    test_data1i.append(D1i[i])
    test_card1i.append(C1i[i])
    
# Reshape data for dimensionality agreement with network 
test_data0  = np.concatenate(test_data0,  0)
test_card0  = np.reshape(test_card0, (-1,1))
test_data0i = np.reshape(np.concatenate(test_data0i, 0),(-1,1))
test_card0i = np.reshape(test_card0i, (-1,1))
test_data1i = np.reshape(np.concatenate(test_data1i, 0),(-1,1))
test_card1i = np.reshape(test_card1i, (-1,1))

with tf.Session() as sess:

    # Initialize
    sess.run(tf.global_variables_initializer())

    # For each epoch
    for ep in range(nb_epoch):
     
        # Compute training accuracy   
        acc = 0
        for i in range(0,num_batches):
            acc += accuracy.eval(feed_dict={label:  chunked_data[i][0], 
                                            data0:  chunked_data[i][1],
                                            card0:  chunked_data[i][2],
                                            data0i: chunked_data[i][3],
                                            card0i: chunked_data[i][4],
                                            data1i: chunked_data[i][5],
                                            card1i: chunked_data[i][6]})
        acc /= num_batches
        
        # Compute test accuracy
        acc_test = accuracy.eval(feed_dict={label:  test_labels, 
                                            data0:  test_data0,
                                            card0:  test_card0,
                                            data0i: test_data0i,
                                            card0i: test_card0i,
                                            data1i: test_data1i,
                                            card1i: test_card1i})
        sys.stdout.write("Epoch %d, Train Accuracy %g %s, Test Accuracy %g %s \n" % 
                         (ep,100*acc,'%',100*acc_test,'%')) 
        
        # Optimize each batch
        for i in range(0,num_batches):
            learning_step.run(feed_dict={label:  chunked_data[i][0], 
                                         data0:  chunked_data[i][1],
                                         card0:  chunked_data[i][2],
                                         data0i: chunked_data[i][3],
                                         card0i: chunked_data[i][4],
                                         data1i: chunked_data[i][5],
                                         card1i: chunked_data[i][6]})

    # Compute final accuracy
    acc_test = accuracy.eval(feed_dict={label:  test_labels, 
                                        data0:  test_data0,
                                        card0:  test_card0,
                                        data0i: test_data0i,
                                        card0i: test_card0i,
                                        data1i: test_data1i,
                                        card1i: test_card1i})
    sys.stdout.write("Test Accuracy = %g %s " % (100*acc_test, '%'))

Epoch 0, Train Accuracy 15.213 %, Test Accuracy 15.1515 % 
Epoch 1, Train Accuracy 42.7566 %, Test Accuracy 42.6263 % 
Epoch 2, Train Accuracy 46.3579 %, Test Accuracy 47.4747 % 
Epoch 3, Train Accuracy 48.523 %, Test Accuracy 49.4949 % 
Epoch 4, Train Accuracy 50.3653 %, Test Accuracy 50.303 % 
Epoch 5, Train Accuracy 51.0812 %, Test Accuracy 51.3131 % 
Epoch 6, Train Accuracy 51.6169 %, Test Accuracy 51.7172 % 
Epoch 7, Train Accuracy 51.8445 %, Test Accuracy 51.9192 % 
Epoch 8, Train Accuracy 52.1406 %, Test Accuracy 52.1212 % 
Epoch 9, Train Accuracy 52.2746 %, Test Accuracy 52.9293 % 
Epoch 10, Train Accuracy 52.6987 %, Test Accuracy 52.5253 % 
Epoch 11, Train Accuracy 52.7433 %, Test Accuracy 52.3232 % 
Epoch 12, Train Accuracy 53.1332 %, Test Accuracy 52.7273 % 
Epoch 13, Train Accuracy 53.1838 %, Test Accuracy 52.7273 % 
Epoch 14, Train Accuracy 53.1451 %, Test Accuracy 52.7273 % 
Epoch 15, Train Accuracy 53.2284 %, Test Accuracy 52.7273 % 
Epoch 16, Train Accuracy 53.5692 %, T